<a href="https://www.kaggle.com/code/sohailelganagy7372/music-genre-classification-using-cnn?scriptVersionId=144012644" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import scipy
import cv2
import os
import pickle
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras


In [ ]:
import os
import librosa
import zipfile

from tensorflow.keras.preprocessing import image
def load_images_from_path(path, label):
    images = []
    labels = []

    for file in os.listdir(path):
        try:
            images.append(image.img_to_array(image.load_img(os.path.join(path, file), target_size=(224, 224, 3))))
            labels.append((label))
        except Exception as e:
            print("Error loading file:")
            print("Error message:")
        
        
    return images, labels

def show_images(images):
    fig, axes = plt.subplots(1, 8, figsize=(20, 20), subplot_kw={'xticks': [], 'yticks': []})

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i] / 255)
        
i=0
data = []
labels=[]
x = []
y = []
for genre in os.listdir("/kaggle/input/photoes/images_original"):
    genre_path = os.path.join("/kaggle/input/photoes/images_original", genre)
    file_count = 0
    images=[]
    #file_path = os.path.join(genre_path, file)
    images, labels = load_images_from_path(genre_path, i)
    #show_images(images)
    x += images
    y += labels
    i=i+1
print(np.array(y).shape)
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.15, random_state=0)

x_train_norm = np.array(x_train) / 255
x_test_norm = np.array(x_test) / 255

y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x_train_norm = preprocess_input(np.array(x_train))
x_test_norm = preprocess_input(np.array(x_test))

train_features = base_model.predict(x_train_norm)
test_features = base_model.predict(x_test_norm)



In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
acc_per_fold = []
loss_per_fold = []
batch_size = 50
img_width, img_height, img_num_channels = 224, 224, 3
no_epochs = 40
verbosity = 1
num_folds = 8
max=0
# Merge inputs and targets
inputs = np.concatenate((train_features, test_features), axis=0)
targets = np.concatenate((y_train_encoded, y_test_encoded), axis=0)
num_folds = 8
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
bestfold=0
bestmodel=0
bestX_train=0
bestY_train=0
bestX_test=0
bestY_test=0
for train, test in kfold.split(inputs, targets):
    
    

  # Define the model architecture
    model = Sequential()
    model.add(Flatten(input_shape=train_features.shape[1:]))
    model.add(Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(10, activation='softmax', kernel_regularizer=keras.regularizers.l2(0.001)))

  # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)
  # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    if((scores[1]*100) > max):
        max=(scores[1]*100)
        bestmodel=model
        bestX_train=inputs[train]
        bestY_train=targets[train]
        bestX_test=inputs[test]
        bestY_test=targets[test]
        bestfold=fold_no
  # Increase fold number
    fold_no = fold_no + 1
    print("best fold is",bestfold)

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:

bestmodel.evaluate(bestX_test,bestY_test)


In [ ]:
pred=bestmodel.predict(bestX_test)
arr2=[]
arr3=[]
for i in range(len(pred)):
    arr=np.array(pred[i])
    arr2.append(arr.argmax())
for i in range(len(pred)):
    arr=np.array(bestY_test[i])
    arr3.append(arr.argmax())
print("predictions",arr2)
print("original",arr3)


In [ ]:
def accuracy(y_true, y_pred):
    c=0
    for i in range(len(y_pred)):
        if(y_pred[i]==y_true[i]):
            c=c+1
    accuracy=c/len(y_pred) 
    return accuracy * 100
print(accuracy(arr2,arr3))